# Enriching leads from a CSV

In this notebook we will use the Sixtyfour API to enrich leads!

First we will import the packages we will need to async fetch results from the Sixtyfour API

In [86]:
import pandas as pd
import aiohttp
import os 
import httpx
from dotenv import load_dotenv
import asyncio
import nest_asyncio
nest_asyncio.apply()
import requests

Next step will load our .env file, make sure to store your api key as SIXTYFOUR_API_KEY

In [110]:
load_dotenv()

True

In [137]:
SIXTYFOUR_API_KEY = os.getenv('SIXTYFOUR_API_KEY')

Let's load the csv file into a pandas dataframe

In [163]:
df = pd.read_csv('berkeley_topology.csv')

In [166]:
df = df.reset_index().drop(columns=['Name'])

In [169]:
df.columns = ['Name', 'Job title', 'Research area', 'Role', 'Research interests']

Next, we will create a struct --- which is basically all the fields we want the api to enrich!

Note: The power of sixtfour is that we can find the most random and custom fields of information on people! This is something which sets us apart from other data providers.

In [170]:
new_fields = {
  "name": "The individual's full name",
  "email": "The individual's email address",
  "phone": "The individual's phone number",
  "linkedin": "LinkedIn URL for the person",
  "location": "The individual's current location",
  "education": "Degrees obtained, institutions attended, and fields of study",
  "research_focus": "Specific areas of research or specialization within topology",
  "publications": "List of relevant publications or research papers",
  "teaching_experience": "Details of teaching experience in relevant courses",
  "software_skills": "Proficiency in mathematical software or programming languages",
  "collaborations": "Notable collaborations with other researchers or institutions",
  "awards": "Awards or recognitions received",
  "professional_memberships": "Memberships in professional organizations",
  "availability": "Current availability and potential start date",
  "is_topology_specialist": "return True or False indicating if the individual specializes in topology"
}


In [171]:
df = df.fillna('')

Next cell of code just parallely pings the api to enrich every row in the dataframe.

In [172]:
async def enrich_lead(
    client: httpx.AsyncClient,
    semaphore: asyncio.Semaphore,
    lead: dict,
    max_retries: int = 3
):
    """Enrich company info using the SixtyFour API with retries and concurrency limits."""
    async with semaphore:
        for attempt in range(max_retries):
            try:
                response = await client.post(
                    "https://api.sixtyfour.ai/enrich-lead",
                    headers={
                        'x-api-key': SIXTYFOUR_API_KEY,
                        "Content-Type": "application/json"
                    },
                    json={
                        "lead_info": lead,
                        "struct": new_fields
                    }
                )
                response.raise_for_status()
                print(f"Response JSON: {str(response.text)[:20]}")
                return response.json()
            except httpx.HTTPStatusError as http_err:
                print(f"HTTP error occurred: {http_err}\nResponse: {response.text}")
            except httpx.RequestError as req_err:
                print(f"Request error occurred: {req_err}")
                if attempt < max_retries - 1:
                    print(f"Retrying ({attempt + 1}/{max_retries})...")
                    await asyncio.sleep(2 ** attempt)
                    continue
            except Exception as e:
                print(f"Unexpected error: {e}")
            return None
    return None

async def process_leads(df: pd.DataFrame) -> list:
    results = []
    semaphore = asyncio.Semaphore(100)
    timeout = httpx.Timeout(300.0, connect=10.0)

    async with httpx.AsyncClient(follow_redirects=True, timeout=timeout) as client:
        tasks = [enrich_lead(client, semaphore, df.iloc[i].to_dict()) for i in range(len(df))]
        results = await asyncio.gather(*tasks, return_exceptions=True)

    return results

In [173]:
results = asyncio.run(process_leads(df))

Response JSON: {"notes":"Ahmad Abas
Response JSON: {"notes":"Gabriel Be
Response JSON: {"notes":"Connor Hal
Response JSON: {"notes":"Robin Huan
Response JSON: {"notes":"Sean Gonza
Response JSON: {"notes":"Hannah Fri
Response JSON: {"notes":"Alan Goldf
Response JSON: {"notes":"Nancy Mae 
Response JSON: {"notes":"Connor Hal
Response JSON: {"notes":"A targeted
Response JSON: {"notes":"Xiaoyu Hua
Response JSON: {"notes":"Liza Jacob
Response JSON: {"notes":"Yueqing Fe
Response JSON: {"notes":"Yueqing Fe
Response JSON: {"notes":"Gabriel Be
Response JSON: {"notes":"Yifan Chen
Response JSON: {"notes":"Ansuman Ba
Response JSON: {"notes":"Alan Goldf
Response JSON: {"notes":"Daigo Ito 
Response JSON: {"notes":"Liza Jacob
Response JSON: {"notes":"Garrett Br
Response JSON: {"notes":"Dennis Che
Response JSON: {"notes":"Xiaoyu Hua
Response JSON: {"notes":"Plan to sc
Response JSON: {"notes":"I have col
Response JSON: {"notes":"Carlos Esp
Response JSON: {"notes":"Initiating
Response JSON: {"notes":"Yif

In [174]:
res = []

for r in results:
    if r == None:
        continue
    d = {}
    d = r['structured_data']
    d['notes'] = r['notes']
    d['findings'] = r['findings']
    res.append(d)

In [175]:
df_enriched = pd.DataFrame(res)

In [187]:
df_enriched[df_enriched['is_topology_specialist'].str.lower().isin(['yes','true'])].reset_index(drop=True).to_csv('berkeley_enriched.csv')

In [191]:
df_enriched[df_enriched['is_topology_specialist'].str.lower().isin(['yes','true'])]

,name,email,phone,linkedin,location,education,research_focus,publications,teaching_experience,software_skills,...,professional_memberships,availability,is_topology_specialist,Name,Job title,Research area,Role,Research interests,notes,findings
0,Ahmad Abassi,,,,"970 Evans Hall, MC 3840, Berkeley, CA 94720",Double bachelor's degrees in Computer Engineer...,Finite-depth standing water waves involving ap...,Co-author of publications on finite-depth stan...,,Implied use of programming in mathematical res...,...,,,true,Ahmad Abassi,Algebra,Geometry/Topology,Graduate Student,,Ahmad Abassi is a graduate student at UC Berke...,[Research includes semi-analytic theory and co...
1,Ansuman Bardalai,bardalai@berkeley.edu,,https://www.linkedin.com/in/ansuman-bardalai-3...,"Berkeley, California, USA","Graduate student, University of California, Be...",Algebra and Geometry/Topology with specializat...,,Teaching assistant for Discrete Mathematics an...,Probable proficiency in mathematical software ...,...,,Active graduate student through 2024-2025 acad...,true,Ansuman Bardalai,Algebra,Geometry/Topology,Graduate Student,"Representation theory, algebraic number theory...",Ansuman Bardalai is a graduate student at the ...,[No publication list available from initial da...
2,Gabriel Beiner,gabriel.beiner@berkeley.edu,(510) 642-6550,,"University of California, Berkeley, CA",Undergraduate degree from University of Toront...,Geometry/Topology including eigenfunctions of ...,"""Failure of L^p Symmetry of Zonal Spherical Ha...",,,...,,Currently a graduate student; exact availabili...,True,Gabriel Beiner,Geometry/Topology,Graduate Student,,,Gabriel Beiner is a graduate student specializ...,[Education: Undergraduate at University of Tor...
3,Aaron Brookner,brookner@math.berkeley.edu,(510) 642-6550,,"1006 Evans Hall, Department of Mathematics, UC...","Graduate student at University of California, ...","Algebra, Geometry/Topology, specialization in ...",At least two publications related to coalgebra...,Served as teaching assistant for Math 55 (Disc...,,...,,,True,Aaron Brookner,Algebra,Geometry/Topology,Graduate Student,Representation theory; quantum groups,Plan to scrape Aaron Brookner's ResearchGate p...,[Confirmed affiliation with University of Cali...
4,Garrett Brown,gmbrown@berkeley.edu,(510) 642-6550,https://www.linkedin.com/in/garrett-brown-1993...,"Berkeley, California, USA","PhD in Mathematics, University of California, ...",Differential geometry and geometric analysis w...,"""The Sign of Scalar Curvature on Kähler Blowup...",Graduate Student Instructor at UC Berkeley sin...,Proficient in programming with background in A...,...,,Limited availability due to international visi...,true,Garrett Brown,Mathematical Analysis,Geometry/Topology,Graduate Student,My interests are in differential geometry and ...,Garrett Brown is a fourth-year Mathematics PhD...,[Research area: Mathematical Analysis focusing...
5,Dennis Chen,dennisfchen@berkeley.edu,(510) 642-6550,https://www.linkedin.com/in/dennis-chen-3b1a4872,"Berkeley, California, United States","PhD in Mathematics, University of California, ...","Deformation theory, homotopy theory, higher ca...",,,,...,,,true,Dennis Chen,Algebra,Geometry/Topology,Graduate Student,"I'm interested in deformation theory, homotopy...",Dennis Chen is a PhD student in Mathematics at...,[Education: Graduate student at UC Berkeley (i...
6,Yifan Chen,yifan-chen@berkeley.edu,(510) 642-6550,,"Berkeley, California, USA",B.S. in Mathematics from Tsinghua University; ...,Geometry and Topology with emphasis on Complex...,"2024 arXiv preprint: ""Calabi-Yau metrics of Ca...",Graduate Student Instructor at UC Berkeley for...,,...,,Currently on the job market for postdoctoral p...,True,Yifan Chen,Algebra,Geometry/Topology,Graduate Student,,Yifan Chen is a graduate student in Mathematic...,[Publication notable paper: arXiv paper on col...
7,Nancy Mae Eagles,nm.eagles@berkeley.edu,,https://www.linkedin.com/in/nancy-mae-eagles-9...,"Berkeley, 

Here's the final result we got!